In [1]:
!pip install boto3 sagemaker


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 861 kB 1.6 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 57 kB 7.0 MB/s             
     |████████████████████████████████| 18.3 MB 3.4 MB/s            
     |████████████████████████████████| 311 kB 83.3 MB/s            
     |████████████████████████████████| 12.4 MB 75.8 MB/s            
     |████████████████████████████████| 82 kB 459 kB/s             
  Using cached PyYAML-6.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (738 kB)
     |████████████████████████████████| 341 kB 83.9 MB/s            
     |████████████████████████████████| 133 kB 80.6 MB/s            
     |████████████████████████████████| 56 kB 9.3 MB/s             
     |████████████████████████████████| 115 kB 81.6 MB/s            
Using legacy 'setup.py install' for sagemaker, since package 'wheel' is not installed.
    Running s

In [9]:
import boto3
import json
from botocore.exceptions import ClientError

# Specify the role name
role_name = 'SageMaker-Role'

# Managed policies for SageMaker
managed_policy_arns = [
    'arn:aws:iam::aws:policy/AmazonSageMakerFullAccess',
    'arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'  # Optional: If your data is in S3
]

# Create an IAM client
iam_client = boto3.client('iam')

# Create the role
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "sagemaker.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }
    
    create_role_response = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_document)
    )
    print("SageMaker role created successfully:", create_role_response['Role']['Arn'])
    
    # Attach managed policies to the role
    for policy_arn in managed_policy_arns:
        iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn=policy_arn
        )
        print(f"Attached policy {policy_arn} to the role.")

except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Role with the same name already exists.")
    else:
        print("Error creating SageMaker role:", e)


SageMaker role created successfully: arn:aws:iam::188775091215:role/SageMaker-Role
Attached policy arn:aws:iam::aws:policy/AmazonSageMakerFullAccess to the role.
Attached policy arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess to the role.


In [11]:
role_arn = create_role_response['Role']['Arn']

In [4]:
role_arn="arn:aws:iam::188775091215:role/SageMaker-Role"

In [5]:
from sagemaker.pytorch import PyTorchModel
# Import packages
import boto3 # AWS Session Management
import sagemaker # SageMaker SDK

model = PyTorchModel(entry_point='inference.py',
                     model_data="s3://lightsketch-models-188775091215/models/model.tar.gz",
                     framework_version='1.12',
                     py_version='py38',
                     role=role_arn
                     )

In [6]:
from sagemaker.deserializers import JSONDeserializer
from datetime import datetime

INSTANCE_TYPE = 'ml.c5.xlarge'
ENDPOINT_NAME = 'yolov8-pytorch-' + str(datetime.utcnow().strftime('%Y-%m-%d-%H-%M-%S-%f'))

predictor = model.deploy(initial_instance_count=1,
                         instance_type=INSTANCE_TYPE,
                         deserializer=JSONDeserializer(),
                         endpoint_name=ENDPOINT_NAME
                         )

----------!

In [11]:
import cv2
import json
import requests

# Replace with your SageMaker endpoint URL
endpoint_url = "https://runtime.sagemaker.us-east-1.amazonaws.com/endpoints/yolov8-pytorch-2023-08-18-22-34-00-476035/invocations"

# Path to the video file
video_path = "/home/ec2-user/ball_tracking/code/inference.py"

# Open the video file
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to JPEG format
    _, img_encoded = cv2.imencode('.jpg', frame)
    img_bytes = img_encoded.tobytes()

    # Set the request headers
    headers = {'Content-Type': 'application/x-image'}

    # Send the inference request to the SageMaker endpoint
    response = requests.post(endpoint_url, data=img_bytes, headers=headers)

    if response.status_code == 200:
        # Parse the JSON response
        result = json.loads(response.content.decode('utf-8'))

        # Process the result as needed
        print(result)
    else:
        print("Inference request failed with status code:", response.status_code)

# Release the video capture
cap.release()


[ERROR:0] global /tmp/pip-req-build-9vck9bv0/opencv/modules/videoio/src/cap.cpp (164) open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.5.4) /tmp/pip-req-build-9vck9bv0/opencv/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): /home/ec2-user/ball_tracking/code/inference.py in function 'icvExtractPattern'


